In [3]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
#s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=16/part-00000-76298f84-5414-49c5-88f6-a981ff06d94d-c000.csv .

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)

date_day = "2020-03-19"
hr = '20'

rrm_local_path = "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-19/hr=20/*.csv


In [4]:
rdd_rrm_local = spark.sparkContext.textFile(rrm_local_path)
rdd_rrm_local.take(3)

['site,ap,ap2,band,time_epoch,rssi',
 '0009dab3-7a6b-4ace-85ff-e6dc20768bcb,5c5b358e297c,"",5,1584651493,-50.0',
 '000cb16d-91d5-42e2-9c1f-a0319a8116bd,5c5b35506699,5c5b3550629d,5,1584651407,-80.0']

In [5]:
# rdd_rrm_local.map(lambda line: (line.split(',')[0], line.split(',')[1])).collect()

In [6]:

df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_local_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [7]:
df_Schema

StructType(List(StructField(site,StringType,true),StructField(ap,StringType,true),StructField(ap2,StringType,true),StructField(band,IntegerType,true),StructField(time_epoch,IntegerType,true),StructField(rssi,DoubleType,true)))

In [8]:
df_scan.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|0009dab3-7a6b-4ac...|5c5b358e297c|        null|   5|1584651493|-50.0|
|000cb16d-91d5-42e...|5c5b35506699|5c5b3550629d|   5|1584651407|-80.0|
|00193f17-4e86-476...|5c5b357eeb7d|5c5b35aed61f|   5|1584651461|-78.0|
|00193f17-4e86-476...|5c5b35aed7e6|5c5b35aed818|  24|1584651462|-59.0|
|00193f17-4e86-476...|5c5b35aed872|5c5b35aed7e6|  24|1584651465|-66.0|
|00193f17-4e86-476...|5c5b35aed87c|5c5b35aed877|  24|1584651466|-75.0|
|00226d33-cd80-4b1...|5c5b35501f9f|5c5b35502003|  24|1584651085|-64.0|
|00252227-5f8f-4e6...|5c5b352e8f25|        null|  24|1584651485|-61.0|
|00272816-8dc2-4f1...|5c5b35ae6dc9|5c5b357f32c7|  24|1584651463|-72.0|
|00458c29-d67b-4cf...|5c5b353e5573|5c5b353e5555|  24|1584651478|-61.0|
|00458c29-d67b-4cf...|5c5b353e565e|5c5b353e579e|   5|1584651499|-67.0|
|00458

In [9]:
df_scan.describe().show()

+-------+--------------------+------------+------------+-----------------+--------------------+------------------+
|summary|                site|          ap|         ap2|             band|          time_epoch|              rssi|
+-------+--------------------+------------+------------+-----------------+--------------------+------------------+
|  count|             2698488|     2698488|     2472892|          2698488|             2698488|           2698488|
|   mean|                null|        null|        null|16.10877943500212|1.5846512685131106E9|-66.59862078319415|
| stddev|                null|        null|        null|9.362791315364358|   630.5341625638516| 8.445860009209433|
|    min|0009dab3-7a6b-4ac...|5c5b350e011c|5c5b350e0090|                5|          1584648000|             -96.0|
|    max|fffb6917-ee0e-45c...|d420b080bb76|d420b080bb76|               24|          1584651598|             -11.0|
+-------+--------------------+------------+------------+-----------------+------

In [10]:
df_scan.count()

2698488

In [11]:
site_id = "d1ee1d22-4b55-4c97-97c4-9d757144f45b"

In [12]:
df_scan_site = df_scan.filter(col('site')==site_id)

In [13]:
df_scan_site.count()

184